In [ ]:
import osmnx as ox
import networkx as nx
import random
import matplotlib.pyplot as plt
import heapq

def dijkstra(graph, start, goal):
    queue = [(0, start)]
    costs = {start: 0}
    parents = {start: None}

    while queue:
        current_cost, current_node = heapq.heappop(queue)

        if current_node == goal:
            path = []
            while current_node is not None:
                path.append(current_node)
                current_node = parents[current_node]
            return path[::-1], costs[goal]

        for neighbor in graph.neighbors(current_node):
            original_weight = graph[current_node][neighbor][0]['length']
            weight = original_weight * random.uniform(0.5, 2.0)
            cost = current_cost + weight

            if neighbor not in costs or cost < costs[neighbor]:
                costs[neighbor] = cost
                parents[neighbor] = current_node
                heapq.heappush(queue, (cost, neighbor))

    return None, float('inf')

def yen_k_shortest_paths(graph, start, goal, k):
    k_paths = []
    k_costs = []

    path, cost = dijkstra(graph, start, goal)
    if path is None:
        return k_paths, k_costs

    k_paths.append(path)
    k_costs.append(cost)

    potential_paths = []

    for i in range(1, k):
        for j in range(len(k_paths[-1]) - 1):
            spur_node = k_paths[-1][j]
            root_path = k_paths[-1][:j + 1]

            removed_edges = []
            for path in k_paths:
                if path[:j + 1] == root_path and len(path) > j + 1:
                    u, v = path[j], path[j + 1]
                    if graph.has_edge(u, v):
                        removed_edges.append((u, v, graph[u][v][0]['length']))
                        graph.remove_edge(u, v)

            spur_path, spur_cost = dijkstra(graph, spur_node, goal)

            if spur_path is not None:
                total_path = root_path[:-1] + spur_path
                total_cost = sum(graph[total_path[n]][total_path[n + 1]][0]['length'] for n in range(len(total_path) - 1))
                heapq.heappush(potential_paths, (total_cost, total_path))

            for u, v, weight in removed_edges:
                graph.add_edge(u, v, length=weight)

        if not potential_paths:
            break

        cost, path = heapq.heappop(potential_paths)
        k_paths.append(path)
        k_costs.append(cost)

    return k_paths, k_costs

def visualize_graphs(graph, start, goal, paths, costs):
    pos = {node: (data['x'], data['y']) for node, data in graph.nodes(data=True)}

    for i, path in enumerate(paths):
        fig, ax = plt.subplots(figsize=(10, 10))
        
        node_colors = ['blue' if node != start and node != goal else 'green' if node == start else 'red' for node in graph.nodes()]
        nx.draw(graph, pos, node_size=10, node_color=node_colors, edge_color='gray', ax=ax)
        nx.draw(graph, pos, nodelist=path, node_size=50, node_color='red', ax=ax)
        nx.draw_networkx_edges(graph, pos, edgelist=list(zip(path, path[1:])), edge_color='red', width=2, ax=ax)

        path_label = f'Path {i + 1}: Cost = {costs[i]:.2f}'
        plt.title(path_label)
        plt.show()

def main():
    place_name = "Hyderabad"
    dist = 1000

    graph = ox.graph_from_address(place_name, dist=dist, network_type='drive')
    nx_graph = nx.MultiDiGraph(graph)

    source = list(nx_graph.nodes())[random.randint(0, len(nx_graph.nodes()) - 1)]
    destination = list(nx_graph.nodes())[random.randint(0, len(nx_graph.nodes()) - 1)]
    while destination == source:
        destination = list(nx_graph.nodes())[random.randint(0, len(nx_graph.nodes()) - 1)]

    print(f"Source: {source}, Destination: {destination}")

    k = 5
    k_paths, k_costs = yen_k_shortest_paths(nx_graph, source, destination, k)
    if k_paths:
        for i, path in enumerate(k_paths):
            print(f"Path {i + 1}: {' -> '.join(map(str, path))}, Cost: {k_costs[i]}")
        visualize_graphs(nx_graph, source, destination, k_paths, k_costs)
    else:
        print("No path found")

if __name__ == "__main__":
    main()